In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import json

import sys
import glob
import pandas as pda
import os
import seaborn as sns
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
import pickle
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import logging
from open_spiel.python.examples.ubc_mccfr_cpp_example import action_to_bids
from open_spiel.python.examples.ubc_nfsp_example import policy_from_checkpoint
from open_spiel.python.examples.ubc_br import BR_DIR, make_dqn_agent
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.examples.ubc_decorators import CachingAgentDecorator
from open_spiel.python.examples.ubc_decorators import CachingAgentDecorator, TakeSingleActionDecorator
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent

from open_spiel.python.pytorch.ubc_nfsp import NFSP
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from open_spiel.python import rl_environment
import yaml
from open_spiel.python.pytorch.ubc_rnn import AuctionRNN
output_notebook()

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [2]:
game_name = 'big_game_2.json'
game = smart_load_sequential_game('clock_auction', dict(filename=game_name))
num_players = 2
num_products = 3
num_actions = game.num_distinct_actions()

In [3]:
input_size = len(game.new_initial_state().child(0).child(0).information_state_tensor())

In [4]:
input_size

3046

In [5]:
model = AuctionRNN(2, 3, input_size, game.num_distinct_actions())
model_gpu = AuctionRNN(2, 3, input_size, game.num_distinct_actions()).to('cuda')

In [6]:
from open_spiel.python.pytorch import ubc_nfsp, ubc_dqn
model = ubc_dqn.MLP(36, [128,], num_actions, 2, 3)
model_gpu = ubc_dqn.MLP(36, [128,], num_actions, 2, 3).to('cuda')

In [7]:
x = game.new_initial_state().child(0).child(0).child(num_actions - 1).child(num_actions - 1).child(num_actions - 1).child(num_actions - 1).information_state_tensor()

In [8]:
batch = model.prep_batch([model.reshape_infostate(x) for _ in range(1024)])
batch_gpu = model_gpu.prep_batch([model_gpu.reshape_infostate(x) for _ in range(1024)]).cuda()

In [9]:
with torch.no_grad():
    for _ in tqdm(range(10000)):
        model(batch)

100%|██████████| 10000/10000 [00:02<00:00, 3828.25it/s]


In [10]:
with torch.no_grad():
    for _ in tqdm(range(10000)):
        model_gpu(batch_gpu)

100%|██████████| 10000/10000 [00:01<00:00, 8854.76it/s]


In [11]:
for _ in tqdm(range(100)):
    model.rnn(batch)

  0%|          | 0/100 [00:00<?, ?it/s]


AttributeError: 'MLP' object has no attribute 'rnn'

In [ ]:
for _ in tqdm(range(100)):
    model_gpu.rnn(batch_gpu)

In [32]:
from open_spiel.python.examples.ubc_nfsp_example import policy_from_checkpoint, main
import pprofile
profiler = pprofile.Profile()
with profiler:
    main(['/apps/open_spiel/open_spiel/python/examples/ubc_nfsp_example.py', '--filename', 'small_game_2.json', '--network_config_file', '/apps/open_spiel/open_spiel/python/examples/notebooks/configs/jan13/lstm_deep.yml', '--output_dir', '/shared/outputs/jan14_small/lstm_creep', '--dispatch_br', 'false', '--num_training_episodes', '10_000', '--iterate_br', 'true', '--report_freq', '2_000', '--device', 'cpu'])

# profiler.print_stats()
profiler.dump_stats("output/profile.txt")


INFO:absl:Iterating best responses
INFO:absl:Loading network parameters from /apps/open_spiel/open_spiel/python/examples/notebooks/configs/jan13/lstm_deep.yml
INFO:absl:Setting numpy and torch seed to 1234
INFO:absl:Network params: {'add_explore_transitions': False, 'anticipatory_param': 0.1, 'batch_size': 256, 'epsilon_end': 0.05, 'epsilon_start': 0.9, 'eval_every': 10000, 'learn_every': 64, 'loss_str': 'mse', 'min_buffer_size_to_learn': 5000, 'num_training_episodes': 10000, 'optimizer_str': 'sgd', 'replay_buffer_capacity': 50000, 'reservoir_buffer_capacity': 2000000, 'rl_learning_rate': 0.01, 'rl_model': 'recurrent', 'rl_model_args': {'hidden_size': 128, 'num_layers': 2, 'rnn_model': 'lstm'}, 'seed': 1234, 'sl_learning_rate': 0.01, 'sl_model': 'recurrent', 'sl_model_args': {'hidden_size': 128, 'num_layers': 2, 'rnn_model': 'lstm'}, 'update_target_network_every': 10000, 'filename': 'small_game_2.json', 'network_config_file': '/apps/open_spiel/open_spiel/python/examples/notebooks/confi

In [27]:
t = torch.tensor([1, 2, 3])

In [29]:
t.unsqueeze(0)

tensor([[1, 2, 3]])

In [8]:
infostate = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8])
idx = torch.tensor([0, 1, 2, 3, 0, 1, 4, 5, 0, 1, 6, 7])
infostate_reshaped = infostate[idx].reshape(3, 4)

In [12]:
len(infostate_reshaped)

3

In [19]:
padded_seq_batch = torch.nn.utils.rnn.pad_sequence([infostate_reshaped], batch_first=True)
sequence_lengths = [len(seq) for seq in [infostate_reshaped]]
packed_seq_batch = torch.nn.utils.rnn.pack_padded_sequence(padded_seq_batch, lengths=sequence_lengths, batch_first=True, enforce_sorted=False)

In [20]:
padded_seq_batch

tensor([[[1, 2, 3, 4],
         [1, 2, 5, 6],
         [1, 2, 7, 8]]])

In [21]:
packed_seq_batch

PackedSequence(data=tensor([[1, 2, 3, 4],
        [1, 2, 5, 6],
        [1, 2, 7, 8]]), batch_sizes=tensor([1, 1, 1]), sorted_indices=tensor([0]), unsorted_indices=tensor([0]))

In [138]:
tr

tensor([[1, 2, 3, 4],
        [1, 2, 5, 6],
        [1, 2, 7, 8]])

In [147]:
def profile():
    for _ in range(1000):
        model_gpu(batch_gpu)

In [148]:
import cProfile, pstats
profiler = cProfile.Profile()
profiler.enable()
profile()
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('tottime')
stats.print_stats()   

         311048 function calls (309048 primitive calls) in 5.641 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    3.414    0.003    4.213    0.004 /apps/open_spiel/open_spiel/python/pytorch/ubc_rnn.py:115(<listcomp>)
   256000    0.799    0.000    0.799    0.000 {method 'unsqueeze' of 'torch._C._TensorBase' objects}
     1000    0.311    0.000    0.311    0.000 {built-in method lstm}
        1    0.186    0.186    5.640    5.640 <ipython-input-147-545d23a2d6f9>:1(profile)
     1000    0.184    0.000    0.184    0.000 {built-in method cat}
     1000    0.162    0.000    0.162    0.000 {method 'unbind' of 'torch._C._TensorBase' objects}
     1000    0.079    0.000    5.437    0.005 /apps/open_spiel/open_spiel/python/pytorch/ubc_rnn.py:101(forward)
     3000    0.078    0.000    0.078    0.000 {method 'index_select' of 'torch._C._TensorBase' objects}
     1000    0.074    0.000    0.074    0.000 {built-in method _p

In [30]:
t = torch.tensor([1, 2, 3]).cuda()

In [32]:
t.detach()

tensor([1, 2, 3], device='cuda:0')

In [14]:
for p in model.parameters():
    print(p.shape)

torch.Size([512, 18])
torch.Size([512, 128])
torch.Size([512])
torch.Size([512])
torch.Size([12, 128])
torch.Size([12])


In [13]:
!pip freeze

absl-py==0.13.0
adjustText==0.7.3
anyio==3.3.0
argon2-cffi==20.1.0
astunparse==1.6.3
async-generator==1.10
attrs==21.2.0
Babel==2.9.1
backcall==0.2.0
bleach==4.0.0
bokeh==2.3.3
cached-property==1.5.2
cachetools==4.2.4
certifi==2021.5.30
cffi==1.14.6
charset-normalizer==2.0.4
clang==5.0
configparser==5.1.0
contextvars==2.4
cycler==0.10.0
dataclasses==0.8
decorator==5.0.9
defusedxml==0.7.1
entrypoints==0.3
flatbuffers==1.12
gast==0.4.0
google-auth==1.35.0
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.41.1
h5py==3.1.0
idna==3.2
immutables==0.16
importlib-metadata==4.6.3
ipykernel==5.5.5
ipython==7.16.1
ipython-genutils==0.2.0
ipywidgets==7.6.5
jedi==0.18.0
Jinja2==3.0.1
joblib==1.0.1
json5==0.9.6
jsonschema==3.2.0
jupyter==1.0.0
jupyter-bokeh==3.0.3
jupyter-client==6.1.12
jupyter-console==6.4.0
jupyter-core==4.7.1
jupyter-server==1.10.2
jupyterlab==3.1.4
jupyterlab-pygments==0.1.2
jupyterlab-server==2.7.0
jupyterlab-widgets==1.0.2
keras==2.6.0
Keras-Preprocessing==1.1.2
kiwiso

In [14]:
!pip uninstall torch

Found existing installation: torch 1.10.0+cpu
Uninstalling torch-1.10.0+cpu:
  Would remove:
    /apps/open_spiel/venv/bin/convert-caffe2-to-onnx
    /apps/open_spiel/venv/bin/convert-onnx-to-caffe2
    /apps/open_spiel/venv/bin/torchrun
    /apps/open_spiel/venv/lib/python3.6/site-packages/caffe2/*
    /apps/open_spiel/venv/lib/python3.6/site-packages/torch-1.10.0+cpu.dist-info/*
    /apps/open_spiel/venv/lib/python3.6/site-packages/torch/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [ ]:
from open_spiel.python.pytorch import ubc_nfsp, ubc_dqn, ubc_rnn
ubc_dqn.AuctionRNN(
   input_size,
   [128,],
   num_actions,
   2, 
   3,
).reshape_infostate(x)